In [21]:
import os
import numpy as np
import pandas as pd
import tempfile
import imageio.v3 as iio
from skimage.io import imread
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from skimage.feature import local_binary_pattern
from radiomics import featureextractor

In [22]:
IMG_DIR = "data/images"
MASK_DIR = "data/predicted_masks"

In [23]:
extractor = featureextractor.RadiomicsFeatureExtractor("radiomics_params.yaml")

In [24]:
intensity_list = []

In [25]:
image_files = sorted(os.listdir(IMG_DIR))
for filename in image_files:
    patient_id = os.path.splitext(filename)[0]
    img_path = os.path.join(IMG_DIR, filename)

    # ---- Find mask ----
    mask_candidates = [f"{patient_id}_mask.png", f"{patient_id}_pred_mask.png"]
    mask_path = None
    for m in mask_candidates:
        p = os.path.join(MASK_DIR, m)
        if os.path.exists(p):
            mask_path = p
            break
    if mask_path is None:
        print(f"No mask found for {patient_id}, skipping.")
        continue

    # ---- Load mask (ensure binary 0/1) ----
    mask_bin = imread(mask_path)
    if mask_bin.ndim == 3:
        mask_bin = mask_bin[:, :, 0]
    mask_bin = np.where(mask_bin > 0, 1, 0)

    # ---- Temporary mask for PyRadiomics ----
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_mask:
        tmp_mask_path = tmp_mask.name
    iio.imwrite(tmp_mask_path, mask_bin.astype(np.uint8))

    # ---- Extract features ----
    features_raw = extractor.execute(img_path, tmp_mask_path, label=1)
    os.remove(tmp_mask_path)

    # ---- Keep only first-order (intensity) features ----
    intensity_features = {k: v for k, v in features_raw.items() if "firstorder" in k}
    intensity_list.append({"ID": patient_id, **intensity_features})

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [26]:
df_intensity = pd.DataFrame(intensity_list)
df_intensity["ID"] = df_intensity["ID"].astype(int)
df_intensity.rename(columns={"ID": "id"}, inplace=True)
LABELS_CSV = "data/labels.csv"

if os.path.exists(LABELS_CSV):
    df_labels = pd.read_csv(LABELS_CSV)
    df_labels["id"] = df_labels["id"].astype(str)
    df_intensity["id"] = df_intensity["id"].astype(str)
    df_combined = pd.merge(df_labels, df_intensity, on="id", how="left")
else:
    df_combined = df_intensity

df_combined.to_csv(LABELS_CSV, index=False)
print(f"Saved/Updated: {LABELS_CSV}")


Saved/Updated: data/labels.csv


In [30]:
df_labels.columns

Index(['id', 'label', 'area', 'perimeter', 'eccentricity', 'solidity',
       'extent', 'major_axis_length', 'minor_axis_length', 'orientation',
       ...
       'LBP_hist_bin_11', 'LBP_hist_bin_12', 'LBP_hist_bin_13',
       'LBP_hist_bin_14', 'LBP_hist_bin_15', 'LBP_hist_bin_16',
       'LBP_hist_bin_17', 'LBP_mean', 'LBP_std', 'LBP_entropy'],
      dtype='object', length=481)

In [27]:
df_intensity.columns

Index(['id', 'original_firstorder_10Percentile',
       'original_firstorder_90Percentile', 'original_firstorder_Energy',
       'original_firstorder_Entropy', 'original_firstorder_InterquartileRange',
       'original_firstorder_Kurtosis', 'original_firstorder_Maximum',
       'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Mean',
       ...
       'wavelet-L_firstorder_Mean', 'wavelet-L_firstorder_Median',
       'wavelet-L_firstorder_Minimum', 'wavelet-L_firstorder_Range',
       'wavelet-L_firstorder_RobustMeanAbsoluteDeviation',
       'wavelet-L_firstorder_RootMeanSquared', 'wavelet-L_firstorder_Skewness',
       'wavelet-L_firstorder_TotalEnergy', 'wavelet-L_firstorder_Uniformity',
       'wavelet-L_firstorder_Variance'],
      dtype='object', length=109)

In [29]:
print("df_labels columns:", df_labels.columns.tolist())
print("df_intensity columns:", df_intensity.columns.tolist())


df_labels columns: ['id', 'label', 'area', 'perimeter', 'eccentricity', 'solidity', 'extent', 'major_axis_length', 'minor_axis_length', 'orientation', 'original_glcm_Autocorrelation', 'original_glcm_ClusterProminence', 'original_glcm_ClusterShade', 'original_glcm_ClusterTendency', 'original_glcm_Contrast', 'original_glcm_Correlation', 'original_glcm_DifferenceAverage', 'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance', 'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Idmn', 'original_glcm_Idn', 'original_glcm_Imc1', 'original_glcm_Imc2', 'original_glcm_InverseVariance', 'original_glcm_JointAverage', 'original_glcm_JointEnergy', 'original_glcm_JointEntropy', 'original_glcm_MCC', 'original_glcm_MaximumProbability', 'original_glcm_SumAverage', 'original_glcm_SumEntropy', 'original_glcm_SumSquares', 'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_GrayLevelNonUniformityNormalized', 'original_glrlm_GrayLevelVariance', 'original_glrlm_HighGrayLevelRunEmphas